In [ ]:
WOS_CREDENTIALS = {
    "username": "admin",
    "password": "********",
    "version": "3.5"
}

In [ ]:
DATAMART_ID = '00000000-0000-0000-0000-000000000000'

In [ ]:
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [ ]:
import os
token = os.environ['USER_ACCESS_TOKEN']

WOS_CREDENTIALS["url"] = os.getenv('RUNTIME_ENV_APSX_URL')
PROJECT_ID = os.getenv('PROJECT_ID')

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "openshift",
   "url": WOS_CREDENTIALS["url"],
   "version": "3.5"
}

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

In [ ]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'], service_instance_id=DATAMART_ID, authenticator=authenticator)
wos_client.version

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
wml_client.set.default_project(PROJECT_ID)

In [ ]:
prod_model_id = None
for model in wml_client.repository.get_details()['models']['resources']:
    if model['metadata']['name'] == 'Attrition 2018 Experiment - P1 LGBMClassifierEstimator':
        prod_model_id = model['metadata']['id']
print(prod_model_id)
prod_model_details = wml_client.repository.get_details(prod_model_id)

In [ ]:
wml_client.repository.download(prod_model_id, filename='attrition_production.zip')

In [ ]:
preprod_model_id = None
for model in wml_client.repository.get_details()['models']['resources']:
    if model['metadata']['name'] == 'Attrition 2020 Experiment - P1 GradientBoostingClassifierEstimator':
        preprod_model_id = model['metadata']['id']
print(preprod_model_id)
preprod_model_details = wml_client.repository.get_details(preprod_model_id)

In [ ]:
wml_client.repository.download(preprod_model_id, filename='attrition_challenger.zip')

## Production model

In [ ]:
SERVICE_PROVIDER_NAME = "Attrition Production Space"
SERVICE_PROVIDER_DESCRIPTION = "Production space for the HR attrition model"

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [ ]:
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == SERVICE_PROVIDER_NAME:
        space_id = space["metadata"]["id"]
        print('Using existing space')
if space_id is None:
    print('Creating new space')
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: SERVICE_PROVIDER_NAME})["metadata"]["id"]

In [ ]:
time.sleep(15)
wml_client.set.default_space(space_id)

In [ ]:
software_spec_uid = wml_client.software_specifications.get_uid_by_name('hybrid_0.1')

meta_props={
 wml_client.repository.ModelMetaNames.NAME: "predictive attrition - 2018",
 wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
 wml_client.repository.ModelMetaNames.TYPE:'wml-hybrid_0.1',
 wml_client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: prod_model_details['entity']['training_data_references'], 
 "schemas": prod_model_details['entity']['schemas'],
}

created_prod_model_details = wml_client.repository.store_model("attrition_production.zip", meta_props)
created_prod_model_id = wml_client.repository.get_model_id(created_prod_model_details)

In [ ]:
meta_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "predictive attrition - 2018",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

production_deployment = wml_client.deployments.create(created_prod_model_id, meta_props = meta_props)
production_deployment_id = wml_client.deployments.get_uid(production_deployment)

In [ ]:
added_service_provider_result = wos_client.service_providers.add(
        name="Attrition Production Space",
        description="",
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = space_id,
        operational_space_id = "production",
        credentials=WML_CREDENTIALS,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=DATAMART_ID, service_provider_id=service_provider_id, deployment_id=production_deployment_id, deployment_space_id = space_id).result['resources'][0]
asset_deployment_details

In [ ]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=DATAMART_ID,service_provider_id=service_provider_id,deployment_id=production_deployment_id,deployment_space_id=space_id)
model_asset_details_from_deployment

In [ ]:
import pandas as pd

df_data_1 = pd.read_csv('/project_data/data_asset/autoai_records_2020.csv')
df_data_1.head()

In [ ]:
feature_columns = df_data_1.drop(['ATTRITION'], axis=1).columns.tolist()
cat_features = []

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=DATAMART_ID,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="ATTRITION",
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                                                                                                    file_name = "data_2019.csv"),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::",
                                                                        "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                                                                        "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                                                                        "iam_url": "https://iam.ng.bluemix.net/oidc/token"}))
        )
    ).result
subscription_id = subscription_details.metadata.id
print('subscription_id: ' + subscription_id)

In [ ]:
time.sleep(20)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

In [ ]:
feed_data = df_data_1.drop(['ATTRITION'], axis=1).sample(n=100)

In [ ]:
fields = feed_data.columns.tolist()
values = feed_data.values.tolist()

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

In [ ]:
predictions = wml_client.deployments.score(production_deployment_id, payload_scoring)
predictions

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

In [ ]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id

## Preproduction model

In [ ]:
SERVICE_PROVIDER_NAME = "Attrition Preproduction Space"
SERVICE_PROVIDER_DESCRIPTION = "Preproduction space for the HR attrition model"

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [ ]:
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == SERVICE_PROVIDER_NAME:
        space_id = space["metadata"]["id"]
        print('Using existing space')
if space_id is None:
    print('Creating new space')
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: SERVICE_PROVIDER_NAME})["metadata"]["id"]

In [ ]:
time.sleep(15)
wml_client.set.default_space(space_id)

In [ ]:
software_spec_uid = wml_client.software_specifications.get_uid_by_name('hybrid_0.1')

meta_props={
 wml_client.repository.ModelMetaNames.NAME: "predictive attrition - challenger",
 wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
 wml_client.repository.ModelMetaNames.TYPE:'wml-hybrid_0.1',
 wml_client.repository.ModelMetaNames.TRAINING_DATA_REFERENCES: preprod_model_details['entity']['training_data_references'], 
 "schemas": prod_model_details['entity']['schemas'],
}

created_preprod_model_details = wml_client.repository.store_model("attrition_challenger.zip", meta_props)
created_preprod_model_id = wml_client.repository.get_model_id(created_preprod_model_details)

In [ ]:
meta_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "predictive attrition - challenger",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

preproduction_deployment = wml_client.deployments.create(created_preprod_model_id, meta_props = meta_props)
preproduction_deployment_id = wml_client.deployments.get_uid(preproduction_deployment)

In [ ]:
added_service_provider_result = wos_client.service_providers.add(
        name="Attrition Preproduction Space",
        description="",
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = space_id,
        operational_space_id = "pre_production",
        credentials=WML_CREDENTIALS,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(
    data_mart_id=DATAMART_ID,
    service_provider_id=service_provider_id,
    deployment_id=preproduction_deployment_id,
    deployment_space_id = space_id).result['resources'][0]
asset_deployment_details

In [ ]:
model_asset_details_from_deployment = wos_client.service_providers.get_deployment_asset(
    data_mart_id=DATAMART_ID,
    service_provider_id=service_provider_id,
    deployment_id=preproduction_deployment_id,
    deployment_space_id=space_id)
model_asset_details_from_deployment

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=DATAMART_ID,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="ATTRITION",
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                                                                                                    file_name = "data_2019.csv"),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::",
                                                                        "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                                                                        "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                                                                        "iam_url": "https://iam.ng.bluemix.net/oidc/token"}))
        )
    ).result
subscription_id = subscription_details.metadata.id
print('subscription_id: ' + subscription_id)

In [ ]:
time.sleep(20)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

In [ ]:
predictions = wml_client.deployments.score(preproduction_deployment_id, payload_scoring)
predictions

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

In [ ]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id

## Evaluate preprod model

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
}
pre_prod_mrm_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id='mrm',
    target=target,
    parameters=parameters
).result

pre_prod_mrm_instance_id = pre_prod_mrm_monitor_details.metadata.id
pre_prod_mrm_instance_id

In [ ]:
def generate_access_token():
    headers={}
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth(WOS_CREDENTIALS["username"], WOS_CREDENTIALS["password"])
    
    ICP_TOKEN_URL= WOS_CREDENTIALS["url"] + "/v1/preauth/validateAuth"
    
    response = requests.get(ICP_TOKEN_URL, headers=headers, auth=auth, verify=False)
    json_data = response.json()
    icp_access_token = json_data['accessToken']
    return icp_access_token

In [ ]:
def upload_and_evaluate(file_name, mrm_instance_id):
    
    print("Running upload and evaluate for {}".format(file_name))
    import json
    import time
    from datetime import datetime

    status = None
    monitoring_run_id = None
    GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES = 32
    GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL = 10
    
    if file_name is not None:
        
        headers = {}
        headers["Content-Type"] = "text/csv"
        headers["Authorization"] = "Bearer {}".format(generate_access_token())
        
        POST_EVALUATIONS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitoring_services/mrm/monitor_instances/{1}/risk_evaluations?test_data_set_name={2}".format(DATAMART_ID, mrm_instance_id, file_name)

        with open(file_name) as file:
            f = file.read()
            b = bytearray(f, 'utf-8')

        response = requests.post(POST_EVALUATIONS_URL, data=bytes(b), headers=headers, verify=False)
        if response.ok is False:
            print("Upload and evalaute for {0} failed with {1}: {2}".format(file_name, response.status_code, response.reason))
            return
        
        headers = {}
        headers["Content-Type"] = "application/json"
        headers["Authorization"] = "Bearer {}".format(generate_access_token())

        GET_EVALUATIONS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/monitoring_services/mrm/monitor_instances/{1}/risk_evaluations".format(DATAMART_ID, mrm_instance_id)
        
        for i in range(GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES):
        
            response = requests.get(GET_EVALUATIONS_URL, headers=headers, verify=False)
            if response.ok is False:
                print("Getting status of upload and evalaute for {0} failed with {1}: {2}".format(file_name, response.status_code, response.reason))
                return

            response = json.loads(response.text)
            if "metadata" in response and "id" in response["metadata"]:
                monitoring_run_id = response["metadata"]["id"]
            if "entity" in response and "status" in response["entity"]:
                status = response["entity"]["status"]["state"]
            
            if status is not None:
                print(datetime.utcnow().strftime('%H:%M:%S'), status.lower())
                if status.lower() in ["finished", "completed"]:
                    break
                elif "error" in status.lower():
                    print(response)
                    break

            time.sleep(GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL)

    return status, monitoring_run_id

In [ ]:
df_data_1 = df_data_1[[c for c in df_data_1 if c not in ['ATTRITION']] + ['ATTRITION']]
df_data_1.head()

In [ ]:
df_data_1.to_csv("test_data_1.csv", encoding="utf-8", index=False)

In [ ]:
upload_and_evaluate("test_data_1.csv", pre_prod_mrm_instance_id)